# Challenge C: Evaulate the hydrogen surface flux from a Eurofer pipe

## Description

Consider hydrogen being transported through a piping network, some may permeate through the pipe walls. For this challenge you will need to evaulate how much hydorgen permeates from the outer surface at steady-state.

Consider a Eurofer pipe with walls 5mm thick at a fixed temperature of 600K.

**📖 Task: Evaulate the hydrogen surface flux from the outer surface of the pipe**

💡Hint: consider the <em>type</em> argument when defining the mesh.

In [2]:
import h_transport_materials as htm

# eurofer properties
D_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(author="chen")[0]

S_eurofer = htm.solubilities.filter(material="eurofer_97").filter(author="chen")[0]

In [1]:
import festim as F


results_folder = "challenge_B/"

my_model = F.Simulation()

# define mesh
size = 5e-03
vertices = np.linspace(0, size, num=1000)

my_model.mesh = F.MeshFromVertices(vertices=vertices, type="cylindrical")

# define materials

# eurofer properties
D_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(author="chen")[0]

S_eurofer = htm.solubilities.filter(material="eurofer_97").filter(author="chen")[0]

eurofer = F.Material(
    id=1,
    D_0=D_eurofer.pre_exp.magnitude,
    E_D=D_eurofer.act_energy.magnitude,
)
my_model.materials = eurofer

# define temperature
my_model.T = F.Temperature(value=600)

# define boundary conditions
my_model.boundary_conditions = [
    F.SievertsBC(
        surfaces=1,
        S_0=S_eurofer.pre_exp.magnitude,
        E_S=S_eurofer.act_energy.magnitude,
        pressure=1e6,
    ),
    F.DirichletBC(surfaces=2, value=0, field="solute"),
]

# define settings
my_model.settings = F.Settings(
    absolute_tolerance=1e-10,
    relative_tolerance=1e-10,
    transient=False,
)

# define exports
my_derived_quantities = F.DerivedQuantities(
    [F.SurfaceFluxCylindrical("solute", surface=2)],
    show_units=True,
)
my_model.exports = F.Exports([my_derived_quantities])

# run simulation
my_model.initialise()
my_model.run()

surface_flux = np.abs(my_derived_quantities[0].data[0])

print(f"Surface flux = {surface_flux:.2e} H/m2/s")

[DESKTOP-6T0UN3L:58786] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)


Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solved problem in 1.80 s
Surface flux = 4.72e+00 H/m2/s
